# An Introduction to Linear Regression

### Notebook created by [Bright Cape](https://brightcape.nl/)
*Author: Maurits Akkerman, Youp Suurmeijer*

# Contents <a name="Contents"></a>

* [Introduction](#Introduction)

* [Step 1: Modelling (Linear Regression)](#Step-1:-Modelling-(Linear-Regression))
    
    - [Multiple Linear Regression](#1.1:-Multiple-Linear-Regression)
    
    - [Feature Selection and Model Comparison](#1.2:-Feature-Selection-and-Model-Comparison)

* [Step 2: Deployment (Conclusion)](#Step-2:-Deployment-(Conclusion))


## Introduction <a name="Introduction"></a>

[[ go back to the top ]](#Contents)

This notebook builts upon the previous [notebook](https://colab.research.google.com/github/MauritsAkkerman/AppliedDataScience/blob/main/Data%20Cleaning%20%26%20Visualization.ipynb) in which we discussed the exploratory analysis, data cleaning, and data understanding.

<img src="https://drive.google.com/uc?id=1yX1hvNeepHnm1O58voKm_qQultGcpNOy" style="width: 800px;" />

In this notebook, the modelling phase is discussed. We focus on predicting a target variable through the use of **linear regression**. The final phase of deployment and subsequent customer acceptance is not executed, though conclusions and next steps are discussed.

This notebook makes use of (a subset of) a public dataset of the Google Merchandise store. This is real-life data from Google Analytics and looks like something you might get from a company when asked to perform data science experiments using their webstore data. Note that therefore the examples might not always look as perfect as you might find in other online materials, but this is real-life data science, and you never get perfect data to start with.

The concept of this notebook is similar to the previous notebook. This notebook also contains normal text with explanations and grey boxes with Python code. To run the Python code, you need to press the play button in the top left corner of the grey box. Again, sometimes you are requested to perform a simple action in order to run the Python code. Perform this action and then press the play button, otherwise the code does not work. If no action is stated, simply press the play button to run the code and move on.

### Motivation
This notebook is specifically focussed on linear regression modelling. But why are we learning linear regression? Because it is:
- widely used
- runs fast
- easy to use (not a lot of tuning required)
- highly interpretable
- a basis for many other methods.

### Import Required Libraries
Let's start by importing the required libraries for this notebook.

In [ ]:
# Import the required libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# And allow plots to appear directly in the notebook.
%matplotlib inline

### Import Cleaned Dataset
Let's import the cleaned dataset of the previous notebook.

In [ ]:
# Download the data. 
# *Note you don't have to understand what happens in this bit of code.
import requests
from io import StringIO

orig_url='https://drive.google.com/file/d/1SJWwLTX6lHKP13A-PudfRnuN7s-_5vxy/view?usp=sharing'
dwn_url='https://drive.google.com/uc?export=download&id=' + orig_url.split('/')[-2]
url = requests.get(dwn_url).text
csv_raw = StringIO(url)

# Read the data into a pandas data frame and show the first 5 rows.
data = pd.read_csv(csv_raw, index_col=0, sep=',')
data['date'] = pd.to_datetime(data['date'],  format='%Y-%m-%d', errors='coerce')
data.head()

# Step 1: Modelling (Linear Regression) <a name="Step-1:-Modelling-(Linear-Regression)"></a>
[[ go back to the top ]](#Contents)

Simple linear regression is an approach used to predict a quantitative response by using a single feature (or "predictor" or "input variable"). It takes the following form:

$ y = β_0 + β_1 \cdot x$

What does each term represent?

- $y$ is the target (or dependent variable)
- $x$ is the feature (or independent variable)
- $β_0$ is the intercept
- $β_1$ is the coefficient for x

Together, $β_0$ and $β_1$ are called the model coefficients. To create your model, you must "learn" the values of these coefficients. And once we've learned these coefficients, we can use the model to predict Sales!

Let's use the library Statsmodels to estimate the model coefficients for the Google Analytics data.

In [ ]:
# Import Statsmodels
import statsmodels.formula.api as smf

# Create a fitted model in one line using the ordinary least squares method.
lm = smf.ols(formula='totalTransactionRevenue ~  pageviews', data=data).fit()

# Print the coefficients
lm.params

### Interpreting Model Coefficients

How do we interpret the 'pageviews' coefficient ($β_1$) of 2.19 shown above?

- A "unit" increase in page views is associated with a 2.19 "unit" increase in totalTransactionRevenue.
- Or more clearly: An additional page view results in 2.19 dollars extra revenue on average.

Note that if an increase in page views was associated with a decrease in sales, $β_1$ would be negative.

### Using the Model for Prediction

Let's say that we have a current customer on our website with already 10 page views, what would we predict for the Sales for that customer by using the coefficients found above?

$y = β_0 + β_1 \cdot x$


$y = 2.38 + 2.19 \cdot 10$


In [ ]:
# Manually calculate the prediction.
# Remove the "#" from the line below and press the play button.
#round(2.380982 + 2.191458 * 10,2)

Thus, we would predict Sales of $24.30 for that customer. 

----------------------------------------------
----------------------------------------------

We can draw more conclusions from these results, namely that the intercept is very high for webstore sales. The intercept of 2.38 means that even with zero pageviews a customer would spend $2.38 dollars. Since a webstore is known to have very low conversion percentages, in the range of 1%-5%, this high intercept is an unlikely result. The high intercept could be explained by several factors. First, outliers are still affecting the results of the linear regression. Second, the conversion rate of the customers in the dataset is much higher than the industry standard.

The latter is the cause in this instance. The dataset provided is balanced, meaning the number of customers that bought an item is equal to the number of customers who did not buy anything. A balanced dataset can be usefull in certain scenario's but in this instance it gives a wrong representation of reality. Therefore, it would lead to the wrong conclusions.

Once again, business and data understanding is critical to any data science project. Without those two, the conclusions drawn from modelling can be misleading.

Nevertheless, we will continu to show how a linear regression model is created to provide an example of the general principles.

----------------------------------------------
----------------------------------------------

In the Python code above we manually calculated the Sales for a customer with 10 pageviews, which resulted in 24.30 dollars.
However, the Python package Statsmodels can also make this prediction itself.

In [ ]:
# Set the value of pageviews to 10.
X_new = pd.DataFrame({'pageviews': [10]})
X_new.head()

In [ ]:
# Use the model to calculate the Sales prediction of this value.
# Fill in the command "lm.predict(X_new)" without apostrophes in the line below and press the play button.


As you can see, we do not have to calculate the Sales manually with the coefficients, but the Statsmodels package calculates the sales of 24.30 dollars for us.

### Plotting the Least Squares Line

Let's make predictions for the smallest and largest observed values of pageviews, and then use the predicted values to plot the least-squares line:

In [ ]:
# Create a DataFrame with the minimum and maximum values of the feature "pageviews".
X_new = pd.DataFrame({'pageviews': [data['pageviews'].min(), data['pageviews'].max()]})
X_new.head()

So, the minimum and maximum value of pageviews are respectively 1 and 89. 

In [ ]:
# Make predictions for those x values and store them by using the Statsmodels package.
preds = lm.predict(X_new)
preds

In [ ]:
# First, plot the observed data.
data.plot(kind='scatter', x='pageviews', y='totalTransactionRevenue')

# Then, plot the least squares line.
plt.plot(X_new, preds, c='red', linewidth=2);

### How well does the model fit the data?

The most common way to evaluate the overall fit of a linear model is by the R-squared value. R-squared is the proportion of variance explained, meaning the proportion of variance in the observed data that is explained by the model. R-squared is between 0 and 1, and higher is better because it means that more variance is explained by the model.

Let's calculate the R-squared value for our simple linear model:


In [ ]:
# Calculate the R-squared value for our linear regression model.
# Remove the "#" from the line below and press the play button.
#round(lm.rsquared,4)

The R-squared is 33.13%. Is that a "good" R-squared value? It's hard to say. The threshold for a good R-squared value depends widely on the domain. Therefore, it's mostly useful as a tool for comparing different models.

## 1.1: Multiple Linear Regression <a name="1.1:-Multiple-Linear-Regression"></a>
[[ go back to the top ]](#Contents)


Simple linear regression can easily be extended to include multiple features. This is called multiple linear regression. So instead of only using the feature pageviews to predict the sales, we use multiple features:

$y= β_0 + β_1 \cdot x_1 + ... +  β_n \cdot x_n$

Each $x$ represents a different feature and each feature has its own coefficient. In this case:

$y = β_0 + β_1 \cdot$ visitNumber $ + β_2 \cdot $ pageviews $ + β_3 \cdot $ timeOnSite $ + β_4 \cdot $ isMobile

Let's make a multiple linear regression model in scikit-learn.

In [ ]:
# Create x and y.
from sklearn.model_selection import train_test_split

# Create a Python list of feature names.
feature_cols = ["visitNumber", "pageviews", "timeOnSite", "isMobile"]

# Use the list to select a subset of the original DataFrame.
all_inputs = data[feature_cols]
all_labels = data["totalTransactionRevenue"]

(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(all_inputs, all_labels, test_size=0.25, random_state=1)

The last line shows the test size of the model is set to 25% (as test_size is set to 0.25). Thus, we have the remaining 75% of the data available to train our regression model, which is equal to 2415 rows of data. The remaining 805 rows are use to test the performance of the trained model.

In [ ]:
# Follow the usual sklearn pattern: import, instantiate, fit.
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(training_inputs, training_classes)

# Print intercept and coefficients.
print(lm.intercept_)
print(lm.coef_)

So, the intercept of our multiple linear regression model is equal to -2.701 and the coefficients are 5.467, 1.884, 0.004 and -12.567.

In [ ]:
# Pair the feature names with the coefficients we just printed to see which coefficient belongs to which feature.
list(zip(feature_cols, lm.coef_))

So, the formula of our current multiple regression model looks like this:

$y = -2.701 + 5.467 \cdot $ visitNumber $ + 1.884 \cdot $ pageviews $ + 0.004 \cdot $ timeOnSite $ - 12.567 \cdot $ isMobile

How do we interpret the timeOnSite coefficient (0.004)?

For a given amount of "visitNumber" and "pageViews", a "unit" increase in "timeOnSite" is associated with a 0.004 "unit" increase in Sales.

Important note:
This is a statement of association, not causation. If an increase in "timeOnSite" was associated with a decrease in sales, $\beta_1$ would be negative.

In [ ]:
# Calculate the R-squared of the multiple linear regression model.
# Remove the "#" from the line below and press the play button.
#round(lm.score(testing_inputs, testing_classes),4)

## 1.2: Feature Selection and Model Comparison <a name="1.2:-Feature-Selection-and-Model-Comparison"></a>
[[ go back to the top ]](#Contents)

Which model gives us the best result, e.g. which combination of features provides the best response? 

In the multiple regression model above we used the four features visitNumber, pageViews, timeOnSite and isMobile to predict our sales. However, sometimes R-squared of the model increases if one of the features is removed.

So, let's see. Does the independent variable timeOnSite belong in our model? In other words, does it improve the quality of our predictions, e.g. does it improve the R-Squared?

Let's remove the feature timeOnSite in the feature_cols below and check the new R-Squared!

In [ ]:
# Create a Python list of feature names, and remove the feature timeOnSite.
feature_cols = ['pageviews', 'visitNumber', 'isMobile']

# Use the list to select a subset of the original DataFrame.
all_inputs = data[feature_cols]

# Split into training and testing sets.
(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(all_inputs, all_labels, test_size=0.25, random_state=1)

# Fit the model to the training data, e.g. learn the coefficients.
lm.fit(training_inputs, training_classes)

# Make predictions on the testing set.
predictions = lm.predict(testing_inputs)

# And finally, calculate the new R-Squared.
round(lm.score(testing_inputs, testing_classes),4)

We see that the new R-Squared value of the model without the feature timeOnSite is 0.3586. Let's stop here, and compare the three models we created so far. In reality you would also investigate how the R-Squared value changes for different combinations of the features.

### Comparing the Models
Based on the R-Squared value we can now compare the three models we created in this notebook.

1. Linear regression with independent variable: "pageviews", R-Squared: 0.3313
3. Multiple linear regression with independent variables: "pageviews", "visitNumber", "isMobile", "timeOnSite", R-Squared: 0.3590
3. Multiple linear regression with independent variables: "pageviews", "visitNumber", "isMobile", R-Squared: 0.3586

The second model has the highest R-Squared and therefore is the preferred option. This example only shows three combinations of variables for multiple linear regression and serves as an example of feature selection. In reality multiple algorithms and concepts exists that structurally apply feature selection to find the best predicting set of variables. These methods are vital when considering tens or hunderds of different prediction variables.

# Step 2: Deployment (Conclusion) <a name="Step-2:-Deployment-(Conclusion)"></a>

[[ go back to the top ]](#Contents)

As mentioned before, this notebook does not delve into the deployment phase. Though, we will revisit the original business case: 

> We're trying to predict the expected revenue (USD) from a webstore customer based on behavioral web statistics

We have performed one cycle of the data science lifecycle and have made a multiple regression model based on the number of page views and on the time spent on the website. Our Model has an R-Squared of 0.3590, but is that sufficient? Remember:

> Our company's Head of Data has no idea of the predictive power of our dataset but has told us to come up with a model with at least an R-squared of 0.6

Hence, the current model does not suffice with the success metric defined. Therefore, in the next iteration we should look to:
1. Include other variables, both from the current dataset as from other databases.
2. Acquire new data, either to fill in missing data or to expand upon the current rows available.
3. Try other modelling approaches.

Due to the extensive reporting on the preprocessing steps undertaken, we can communicate these with stakeholders to improve the incoming data for future projects.


### Machine learning notebook
In the next notebook we will apply some machine learning algorithms to the same dataset to see if they can outperform multiple linear regression.

* [Machine Learning Algorithms](https://colab.research.google.com/github/MauritsAkkerman/AppliedDataScience/blob/main/Machine%20Learning%20(Supervised%20Classification)%20Case.ipynb)
    
And if you want to look back at the data cleaning steps performed:

* [Data Cleaning](https://colab.research.google.com/github/MauritsAkkerman/AppliedDataScience/blob/main/Data%20Cleaning%20%26%20Visualization.ipynb)